In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import zoom, gaussian_filter
from skimage.filters import threshold_otsu

import os

In [ ]:
def apply_nonlinear_transformation(field, gamma):

    adjusted_field = np.maximum(field, 0)
    adjusted_field = np.power(adjusted_field, gamma)

    return adjusted_field / adjusted_field.max()

In [ ]:
def segment_vessel_otsu(velocity_field, pressure_field, gamma=1.0, sigma=1.0):

    velocity_magnitude = np.linalg.norm(velocity_field, axis=2)
    velocity_transformed = apply_nonlinear_transformation(velocity_magnitude, gamma)
    pressure_transformed = apply_nonlinear_transformation(pressure_field, gamma)

    blurred_pressure = gaussian_filter(pressure_transformed, sigma=sigma)
    blurred_velocity = gaussian_filter(velocity_transformed, sigma=sigma)

    velocity_threshold = threshold_otsu(blurred_velocity)
    pressure_threshold = threshold_otsu(blurred_pressure)

    velocity_mask = (blurred_velocity > velocity_threshold/1.1).astype(np.float32)
    pressure_mask = (blurred_pressure > pressure_threshold).astype(np.float32)
    combined_mask = np.maximum(velocity_mask, pressure_mask)

    return combined_mask

In [ ]:
def sample_points_from_mask(mask, num_samples=100):

    y_coords, x_coords = np.where(mask == 1)
    indices = np.random.choice(len(x_coords), num_samples, replace=False)
    sampled_points = [(x_coords[i], y_coords[i]) for i in indices]

    return sampled_points